In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# --- 1. VERİ SETİ (Örnek: Film Yorumları ve 1-5 arası Puanlar) ---
# Not: PyTorch'ta sınıflar 0'dan başlar. 
# 1 Yıldız -> Etiket 0
# 5 Yıldız -> Etiket 4
data = [
    ("bu film harika bayıldım", 4),       # 5 Yıldız
    ("mükemmel bir senaryo çok iyi", 4),  # 5 Yıldız
    ("idare eder fena değil", 2),         # 3 Yıldız
    ("vakit kaybı sakın izlemeyin", 0),   # 1 Yıldız
    ("kötü oyunculuk ve sıkıcı", 0),      # 1 Yıldız
    ("biraz uzun ama güzeldi", 3)         # 4 Yıldız
]

# --- 2. ÖN İŞLEME (Vocabulary Oluşturma ve Padding) ---
# Basit kelime sözlüğü oluşturma
word_to_ix = {"<PAD>": 0} # Padding için 0. indexi ayırıyoruz
for sent, _ in data:
    for word in sent.split():
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

vocab_size = len(word_to_ix)
output_size = 5  # Senin Keras modelindeki Dense(5)
embedding_dim = 10
max_len = 6      # Cümleleri bu uzunluğa sabitleyeceğiz

def make_vector(sentence, word_to_ix, max_len):
    # Cümleyi indekslere çevir
    vec = [word_to_ix.get(word, 0) for word in sentence.split()]
    # Padding (kısa cümleleri 0 ile doldur, uzunları kırp)
    if len(vec) < max_len:
        vec += [0] * (max_len - len(vec))
    else:
        vec = vec[:max_len]
    return vec

# Veriyi Tensor'a çevirme
inputs = []
labels = []
for sent, label in data:
    inputs.append(make_vector(sent, word_to_ix, max_len))
    labels.append(label)

inputs = torch.LongTensor(inputs)  # Model girdisi (ams)
labels = torch.LongTensor(labels)  # Beklenen sonuçlar

# --- 3. MODEL TANIMI (Önceki Kodun Aynısı) ---
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size=32, output_size=5):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.gru = nn.LSTM(embedding_size, hidden_size, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(hidden_size * 2, output_size)
        # Not: Eğitimde CrossEntropyLoss kullanacağımız için Softmax'i burada kullanmıyoruz.
        # Loss fonksiyonu kendi içinde hallediyor.

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.gru(x)
        out = out[:, -1, :] # Son zaman adımını al
        x = self.dropout(out)
        x = self.fc(x)
        return x

model = TextClassifier(vocab_size, embedding_dim, output_size=output_size)

# --- 4. EĞİTİM DÖNGÜSÜ ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

print("Eğitim başlıyor...")
for epoch in range(50):  # 50 Epoch eğitim
    model.train()
    optimizer.zero_grad()
    
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# --- 5. TAHMİN (TEST) ---
print("\n--- Test Sonuçları ---")
def predict_sentiment(sentence):
    model.eval()
    vec = torch.LongTensor([make_vector(sentence, word_to_ix, max_len)])
    with torch.no_grad():
        output = model(vec)
        # Softmax uygulayarak olasılıkları görelim
        probs = torch.nn.functional.softmax(output, dim=1)
        predicted_class = torch.argmax(probs).item()
    return predicted_class, probs[0]

test_sentences = ["bu film harika", "çok sıkıcı ve kötü"]

for sent in test_sentences:
    pred_class, probs = predict_sentiment(sent)
    print(f"Cümle: '{sent}'")
    print(f"Tahmin Edilen Puan (0-4 arası): {pred_class}")
    print(f"Olasılıklar: {probs.numpy()}")
    print("-" * 30)

Eğitim başlıyor...
Epoch 10, Loss: 1.3744
Epoch 20, Loss: 0.4889
Epoch 30, Loss: 0.0743
Epoch 40, Loss: 0.0285
Epoch 50, Loss: 0.0096

--- Test Sonuçları ---
Cümle: 'bu film harika'
Tahmin Edilen Puan (0-4 arası): 4
Olasılıklar: [4.0343651e-04 7.6196055e-05 2.4040686e-03 1.3896044e-04 9.9697733e-01]
------------------------------
Cümle: 'çok sıkıcı ve kötü'
Tahmin Edilen Puan (0-4 arası): 0
Olasılıklar: [9.9456292e-01 2.9861338e-05 7.7452528e-04 3.9873752e-03 6.4528512e-04]
------------------------------
